# Analyze what classes labels are predicted for sequences with man walking with a ladder

Those sequences should probably be the easiest to classify.

In [1]:
import pandas as pd
from pathlib import Path

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import numpy as np
import torch

from ilids.towhee_utils.override.movinet import read_kinetics_600_classmap

In [2]:
# cell has parameters tag for papermill execution
movinet_variation: str = "movineta0"
normalize_features: str = False
topk: int = 5

In [3]:
print(f"Using variation: {movinet_variation}")
print(f"Normalize features: {normalize_features}")

Using variation: movineta0
Normalize features: False


In [4]:
# Constants
SOURCE_PATH = Path().resolve() # this template folder

SUBJECT_APPROACH_TYPE = "Walk With Ladder" # looking at features of the sequences of this type

## Loading Ground Truth and Experiment Results

In [5]:
# Load the ground truth sequences
tp_fp_sequences_path = (
    SOURCE_PATH.parent.parent / "data" / "handcrafted-metadata" / "tp_fp_sequences.csv"
)
# the first column being the sequence file name: e.g. "SZTEA101a_00_05_37.mov"
SEQUENCES_DF = pd.read_csv(tp_fp_sequences_path, index_col=0)
# Only keep relevant columns
SEQUENCES_DF = SEQUENCES_DF[
    [
        # "Classification", All sequences are "TP" when filtering only the "Walk With Ladder" type
        "Distance",
        "SubjectApproachType",
        "SubjectDescription",
        "Stage",
    ]
]
# Prefix the index with the path to the sequence file
SEQUENCES_DF = SEQUENCES_DF.set_index("data/sequences/" + SEQUENCES_DF.index)

## Load the pickle result features

In [6]:
# Load pickle results
pickle_file = SOURCE_PATH / f"{movinet_variation}.pkl"
features_df = pd.read_pickle(pickle_file)

# MoViNet produce 600 features (the 600 classes of the Kinetics-600 dataset)
FEATURES_COLUMNS_INDEXES = pd.RangeIndex.from_range(range(600))

if normalize_features:
    features = features_df[FEATURES_COLUMNS_INDEXES].to_numpy()
    features_df[FEATURES_COLUMNS_INDEXES] = (features / np.linalg.norm(features, axis=-1, keepdims=True)).tolist()

df = SEQUENCES_DF.join(features_df)

In [7]:
# Keep only the sequences matching the subject approach type
df = df[df["SubjectApproachType"] == SUBJECT_APPROACH_TYPE].drop(columns=["SubjectApproachType"])
len(df)

26

## Compute their Top K scores and labels

In [8]:
softmax = torch.nn.Softmax(dim=1)

In [9]:
# initialize the class map
classmap = read_kinetics_600_classmap()

In [10]:
features = torch.tensor(df[FEATURES_COLUMNS_INDEXES].values) # N, 600 (600 being the number of classes in Kinetics-600)

probabilities = softmax(features) # N, 600
predictions_scores, predictions_classes = probabilities.topk(topk) # N, topk

In [11]:
# labels = [[classmap[int(i)] for i in video_pred_classes] for video_pred_classes in predictions_classes]  # list of string with topk elements
# scores = [[round(float(x), 5) for x in video_pred_scores] for video_pred_scores in predictions_scores]  # float percentages (e.g. 0.34 -> 34%; sum doesn't sum to 1, as topk from preds)

In [12]:
# Produce heatmap with the topk predictions
# x axis: classes
all_prediction_classes = predictions_classes.ravel()
all_prediction_classes.sort()
all_prediction_classes = all_prediction_classes.unique()

all_prediction_classes_names = [classmap[int(i)] for i in all_prediction_classes]
# y axis: sequences
# color: score
#  -> for each sequences
#    -> for each remaining predicted classes
#      -> if the predicted class is in the topk, then the score is the score of the prediction
#      -> else the score is 0
#  -> sum the scores for each sequences
#  -> normalize the scores by the number of sequences
#  -> produce the heatmap
z = [
    [
        scores[video_pred_classes == predicted_class].item()
        if predicted_class in video_pred_classes 
        else 0
        for predicted_class in all_prediction_classes
    ]
    for video_pred_classes, scores in zip(predictions_classes, predictions_scores)
]
# reduce z to the mean of the scores
classes_means = np.array(z).mean(axis=0)

In [13]:
fig = make_subplots(
    rows=2, cols=1,
    row_heights=[0.15, 0.85],
    vertical_spacing=0.02,
)

# add mean of scores as bars
fig.add_trace(
    go.Bar(
        x=all_prediction_classes_names, 
        y=classes_means,
        marker_color=classes_means,
        marker_colorscale='dense',
    ),
    row=1, col=1,
)
# hide x axis of the bar plot
fig.update_xaxes(showticklabels=False, row=1, col=1)

# add heatmap
fig.add_trace(
    go.Heatmap(
        z=z, 
        x=all_prediction_classes_names,
        y=df.index,
        colorscale='dense',
    ),
    row=2, col=1,
)

fig.update_layout(height=800, title_text="Topk predictions heatmap")

fig.show()

In [14]:
# Print the best working classes
mean_score_df = pd.DataFrame(classes_means, index=pd.Index(all_prediction_classes_names, name="class"), columns=["score"])
mean_score_df["model_name"] = movinet_variation

print(
    mean_score_df
        .sort_values(by="score", ascending=False)
        .head(15)
        .to_csv()
)

class,score
golf putting,0.0714001223588219
stretching arm,0.065495123679284
pirouetting,0.0510407001666653
slapping,0.040222761865991816
playing didgeridoo,0.040152427095633283
finger snapping,0.03850344735502194
shaking head,0.0383991369834313
headbutting,0.0366574486600681
presenting weather forecast,0.03573111602320121
golf chipping,0.02996674137046704
using a paint roller,0.022471194083874043
shooting goal (soccer),0.019069109326944902
stretching leg,0.018888531539302606
golf driving,0.012429973826958584
hurling (sport),0.01141866840995275

